# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns
sns.set()

# dataset - https://www.kaggle.com/mkechinov/ecommerce-events-history-in-cosmetics-shop?select=2019-Nov.csv

df=pd.read_csv('C:\\Users\\MAYBHOSA\\Downloads\\2019-Nov\\2019-Nov.csv')

# Data Pre-Processing

In [2]:
df.shape

(4635837, 9)

In [3]:
df=df[df['event_type'] != 'view']
df.shape

(2559705, 9)

# Finding Unique No. of Categories

In [4]:
unique_categories=df['category_id'].value_counts()
unique_categories

1487580007675986893    182576
1487580005595612013    136148
1487580005671109489    100861
1487580006317032337     97394
1487580005092295511     90513
                        ...  
1487580013564788937         4
2195085258339123402         3
1487580011383750769         3
1487580008128971748         2
1487580009739583530         2
Name: category_id, Length: 451, dtype: int64

In [5]:
cart_users=df[(df['event_type'] == 'cart')].user_id.unique()
cart_users

array([553329724, 564506666, 514649199, ..., 523425698, 252311454,
       448212838], dtype=int64)

In [49]:
df= df[(df['user_id'].isin(cart_users))]
df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
1,2019-11-01 00:00:09 UTC,cart,5844397,1487580006317032337,NaN,NaN,2.38,553329724,2067216c-31b5-455d-a1cc-af0575a34ffb
3,2019-11-01 00:00:11 UTC,cart,5876812,1487580010100293687,NaN,jessnail,3.16,564506666,186c1951-8052-4b37-adce-dd9644b1d5f7
4,2019-11-01 00:00:24 UTC,remove_from_cart,5826182,1487580007483048900,NaN,NaN,3.33,553329724,2067216c-31b5-455d-a1cc-af0575a34ffb
5,2019-11-01 00:00:24 UTC,remove_from_cart,5826182,1487580007483048900,NaN,NaN,3.33,553329724,2067216c-31b5-455d-a1cc-af0575a34ffb
8,2019-11-01 00:00:34 UTC,remove_from_cart,5870838,1487580007675986893,NaN,milv,0.79,429913900,2f0bff3c-252f-4fe6-afcd-5d8a6a92839a
...,...,...,...,...,...,...,...,...,...
4635824,2019-11-30 23:59:24 UTC,remove_from_cart,5608703,1487580005553668971,NaN,NaN,9.52,566513990,b5053009-b3c4-4581-b05f-6199d1a2037e
4635826,2019-11-30 23:59:25 UTC,remove_from_cart,5810080,1487580005268456287,NaN,irisk,6.51,566513990,b5053009-b3c4-4581-b05f-6199d1a2037e
4635827,2019-11-30 23:59:25 UTC,remove_from_cart,5810080,1487580005268456287,NaN,irisk,6.51,566513990,b5053009-b3c4-4581-b05f-6199d1a2037e
4635828,2019-11-30 23:59:27 UTC,remove_from_cart,5801131,1487580005486560104,NaN,irisk,3.65,566513990,b5053009-b3c4-4581-b05f-6199d1a2037e


In [7]:
def customer_cart_abondan(user_id):
    user_df= df[df['user_id']== user_id]

    added_to_cart=user_df[user_df['event_type']=='cart']
    added_to_cart=added_to_cart[['product_id','category_id']]
    removed_from_cart=user_df[user_df['event_type']=='remove_from_cart'].product_id

    

    left_in_cart=added_to_cart[~added_to_cart['product_id'].isin(removed_from_cart)]
    product_added_per_category=left_in_cart.groupby('category_id').count().reset_index()
    #print('left_in_cart\n',left_in_cart)

    purchased=user_df[user_df['event_type']=='purchase'].product_id
    #print('purchased\n',purchased)

    abondaned=left_in_cart[~left_in_cart['product_id'].isin(purchased)]
    product_abondaned_per_category=abondaned.groupby('category_id').count().reset_index()
    print('abondaned\n',abondaned)


    merged_df = pd.merge(product_added_per_category,product_abondaned_per_category, on='category_id')

    return merged_df.values.tolist()

In [9]:
import time
begin=time.time()
arr=[]
a=0
for i in cart_users:
    arr.extend(customer_cart_abondan(i))
    a=a+1
    print(a)
    if(a==200):
        break
end=time.time()
print(end-begin)


abondaned
        product_id          category_id
61344     5846600  1842735760499802745
1
abondaned
     product_id          category_id
3      5876812  1487580010100293687
29     5862929  1487580010100293687
2
abondaned
          product_id          category_id
4422424     5887254  1977575787259232875
4424669     5823667  1487580008246412266
4424845     5819894  1783999067156644376
4424876     5808547  1487580008313521133
3
abondaned
          product_id          category_id
1607086        5013  1487580008145748965
1607133     5549834  1487580007281722301
1607284     5809921  1487580010100293687
1607314     5749149  1487580007592100809
1607397     5749150  1487580007592100809
1607501     5620922  1487580007306888126
1607763     5885842  1487580007910867929
2304738     5561064  1487580005595612013
2304756     5550028  1487580005595612013
2304853     5561064  1487580005595612013
2307815     5549842  1487580005268456287
4
abondaned
        product_id          category_id
97873     58577

In [10]:
arr

[[1842735760499802745, 1, 1],
 [1487580010100293687, 2, 2],
 [1487580008246412266, 1, 1],
 [1487580008313521133, 1, 1],
 [1783999067156644376, 1, 1],
 [1977575787259232875, 1, 1],
 [1487580005268456287, 1, 1],
 [1487580005595612013, 4, 3],
 [1487580007281722301, 1, 1],
 [1487580007306888126, 1, 1],
 [1487580007592100809, 2, 2],
 [1487580007910867929, 1, 1],
 [1487580008145748965, 1, 1],
 [1487580010100293687, 1, 1],
 [1487580008246412266, 1, 1],
 [1783999073758478650, 1, 1],
 [1487580009026551821, 1, 1],
 [1487580005268456287, 1, 1],
 [2193074740552270669, 1, 1],
 [1487580006895846315, 1, 1],
 [1783999063314661546, 1, 1],
 [1487580011585077370, 4, 4],
 [1487580011627020412, 9, 9],
 [1891434351850160381, 3, 3],
 [1487580005595612013, 2, 2],
 [1487580007910867929, 1, 1],
 [1487580008816836611, 1, 1],
 [1487580004857414477, 23, 23],
 [1487580011283087468, 1, 1],
 [1487580011534745720, 3, 3],
 [1487580011585077370, 3, 2],
 [1487580013212467384, 2, 2],
 [1487580005092295511, 1, 1],
 [148758

In [11]:
dff=pd.DataFrame(arr,columns=['category_id','added','abondand'])
dff

,category_id,added,abondand
0,1842735760499802745,1,1
1,1487580010100293687,2,2
2,1487580008246412266,1,1
3,1487580008313521133,1,1
4,1783999067156644376,1,1
...,...,...,...
599,1487580006317032337,11,11
600,1487580009261432856,1,1
601,1487580009311764506,3,3
602,1487580011408916594,6,6


In [39]:
dff=dff.groupby('category_id').sum()
dff

,added,abondand,rate
category_id,,,
1487580004832248652,1,1,100.0
1487580004857414477,25,25,100.0
1487580004882580302,1,1,100.0
1487580004916134735,6,6,100.0
1487580005008409427,2,1,50.0
...,...,...,...
2151191070984110951,1,1,100.0
2151191071118328683,1,1,100.0
2193074740552270669,6,6,100.0


In [45]:
print (arr)
"""a = pd.DataFrame(arr,  columns=['user_id','category_id', 'rate'])
a"""

[[1842735760499802745, 1, 1], [1487580010100293687, 2, 2], [1487580008246412266, 1, 1], [1487580008313521133, 1, 1], [1783999067156644376, 1, 1], [1977575787259232875, 1, 1], [1487580005268456287, 1, 1], [1487580005595612013, 4, 3], [1487580007281722301, 1, 1], [1487580007306888126, 1, 1], [1487580007592100809, 2, 2], [1487580007910867929, 1, 1], [1487580008145748965, 1, 1], [1487580010100293687, 1, 1], [1487580008246412266, 1, 1], [1783999073758478650, 1, 1], [1487580009026551821, 1, 1], [1487580005268456287, 1, 1], [2193074740552270669, 1, 1], [1487580006895846315, 1, 1], [1783999063314661546, 1, 1], [1487580011585077370, 4, 4], [1487580011627020412, 9, 9], [1891434351850160381, 3, 3], [1487580005595612013, 2, 2], [1487580007910867929, 1, 1], [1487580008816836611, 1, 1], [1487580004857414477, 23, 23], [1487580011283087468, 1, 1], [1487580011534745720, 3, 3], [1487580011585077370, 3, 2], [1487580013212467384, 2, 2], [1487580005092295511, 1, 1], [1487580006317032337, 1, 1], [1487580006

"a = pd.DataFrame(arr,  columns=['user_id','category_id', 'rate'])\na"

In [13]:
final_df= pd.DataFrame(arr, columns=['user_id','aboundand_items', 'items_left_after_remove'])
final_df

,user_id,aboundand_items,items_left_after_remove
0,1842735760499802745,1,1
1,1487580010100293687,2,2
2,1487580008246412266,1,1
3,1487580008313521133,1,1
4,1783999067156644376,1,1
...,...,...,...
599,1487580006317032337,11,11
600,1487580009261432856,1,1
601,1487580009311764506,3,3
602,1487580011408916594,6,6


# Calculating Cart Abandonment Rate Per Category

In [14]:
dff['rate'] = ((dff['abondand']*100)/dff['added'])
dff.sort_values('rate', ascending=True)

,added,abondand,rate
category_id,,,
1487580005008409427,2,1,50.000000
1487580011585077370,23,15,65.217391
1487580012096782476,3,2,66.666667
1487580005671109489,77,55,71.428571
2084144451428549153,12,10,83.333333
...,...,...,...
1487580009076883471,2,2,100.000000
1487580009143992338,5,5,100.000000
1487580009202712597,1,1,100.000000


# Average Cart Abandonment Rate Per Category

In [28]:
dff['rate'].mean()

99.00168758833357